In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

In [21]:
# Definir el rango de fechas
start_date = datetime(2016, 1, 1)  # Fecha de inicio
end_date = datetime(2025, 3, 27)   # Fecha de término

# Lista para almacenar los datos
data = []

In [22]:
# Recorrer cada día en el rango de fechas
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")  # Formato YYYYMMDD
    url = f"https://www.sismologia.cl/sismicidad/catalogo/{current_date.year}/{current_date.strftime('%m')}/{date_str}.html"
    
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", class_="sismologia detalle")  # Buscar la tabla específica
        
        if table:
            rows = table.find_all("tr")
            if len(rows) > 1:
                for row in rows[1:]:  # Omitir encabezado
                    cols = row.find_all("td")
                    if len(cols) >= 5:
                        fecha_local = cols[0].text.strip()
                        fecha_utc = cols[1].text.strip()
                        
                        # Extraer Latitud y Longitud correctamente
                        latitud = None
                        longitud = None
                        if len(cols) > 2:
                            lat_long_text = cols[2].text.strip()
                            lat_long_split = lat_long_text.split(" ")  # Separar por espacio
                            latitud = lat_long_split[0] if len(lat_long_split) > 1 else None
                            longitud = lat_long_split[1] if len(lat_long_split) > 1 else None
                        
                        profundidad = cols[3].text.strip()
                        magnitud = cols[4].text.strip()
                        
                        data.append([fecha_local, fecha_utc, latitud, longitud, profundidad, magnitud])
            else:
                print(f"No hay datos en la tabla para {date_str}")
        else:
            print(f"No se encontró la tabla para {date_str}")
    else:
        print(f"Error al acceder a {url}, Código: {response.status_code}")
    
    current_date += timedelta(days=1)

No hay datos en la tabla para 20161231
No hay datos en la tabla para 20171231
No hay datos en la tabla para 20181231
No hay datos en la tabla para 20191231


In [23]:
# Guardar en un archivo CSV si hay datos
columns = ["Fecha Local", "Fecha UTC", "Latitud", "Longitud", "Profundidad (km)", "Magnitud"]
df = pd.DataFrame(data, columns=columns)
if not df.empty:
    df.to_csv("sismos.csv", index=False, encoding="utf-8-sig")
    print("Datos guardados en sismos.csv")
else:
    print("No se encontraron datos de sismos en el período seleccionado.")


Datos guardados en sismos.csv


In [24]:
df = pd.read_csv('../data/sismos.csv')
df

,Fecha Local,Fecha UTC,Latitud,Longitud,Profundidad (km),Magnitud
0,2016-01-01 20:55:03107 km al SE de OllagÃ¼e,2016-01-01 23:55:03,-22.040,-67.695,221 km,3.3 Ml
1,2016-01-01 18:59:5277 km al S de OllagÃ¼e,2016-01-01 21:59:52,-21.901,-68.414,132 km,3.3 Ml
2,2016-01-01 18:18:2853 km al O de Los Vilos,2016-01-01 21:18:28,-31.877,-72.066,15 km,3.5 Ml
3,2016-01-01 16:00:3733 km al O de Canela Baja,2016-01-01 19:00:37,-31.417,-71.802,15 km,3.3 Ml
4,2016-01-01 14:53:5930 km al SO de Canela Baja,2016-01-01 17:53:59,-31.522,-71.736,38 km,2.8 Ml
...,...,...,...,...,...,...
69128,2025-03-26 23:59:2543 km al S de Huasco,2025-03-27 02:59:25,-28.829,-71.367,34 km,2.6 Ml
69129,2025-03-26 23:44:3335 km al NE de Pica,2025-03-27 02:44:33,-20.226,-69.147,91 km,2.7 Ml
69130,2025-03-26 22:48:3859 km al SE de Socaire,2025-03-27 01:48:38,-24.018,-67.552,252 km,2.6 Ml
69131,2025-03-26 22:12:2634 km al NE de Farellones,2025-03-27 01:12:26,-33.072,-70.153,112 km,2.6 Ml
